In [1]:
# https://www.mdpi.com/search?advanced=(@(abstract)BIM@(abstract)Green%20Buildings)

In [2]:
# Requires Uni environment 

In [3]:
# pip install selenium

In [4]:
import requests
from bs4 import BeautifulSoup
from pandas.io.html import read_html
import pandas as pd
import re

In [5]:
df = pd.DataFrame()
df.to_csv('blankFile.csv')

columnNames = ['searchTerm1','searchTerm2','url','Title']

df = pd.read_csv("blankFile.csv", names= columnNames , header = None)
df = df.iloc[1:]

for i in range(20):
    section = "section" + str(i)
    sectionData = "sectionData" + str(i)
    df[section] = ''
    df[sectionData] = ''

df

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19


In [6]:
# searchTerm1 = ['BIM', 'Building Information Modelling']
# searchTerm2 = ['Green Building', 'Green Buildings']

# searchTerm1 = ['a']
# searchTerm2 = ['green']



tc = 0

searchTerm1 = ['BIM' , 'Building Information Modelling','Building information model','6D BIM', 'Digital construction','Smart construction','Digital engineering']
searchTerm2 = ['Green Building' , 'Green Buildings','Sustainable building','Sustainable buildings','Environmentally friendly design' , 'Sustainability' , 'Sustainable development' , 'Energy','Water' ,'Waste' , 'Thermal comfort','Electricity' ,'Solar','Wind power' , 'Green infrastructure','Life cycle assessment']

counter = 0
programSearchTerms1 = []
programSearchTerms2 = []

for term2 in searchTerm2:
    text2 = ""
    terms2 = term2.split(' ')
    for t2 in terms2:
        text2 = text2 + t2 + "+"

    fileIndex = text2.rfind("+")
    te = text2[:fileIndex]
    programSearchTerms2.append(te)

print(programSearchTerms2)

for term1 in searchTerm1:
    text1 = ""
    terms1 = term1.split(' ')
    for t2 in terms1:
        text1 = text1 + t2 + "+"

    fileIndex = text1.rfind("+")
    te = text1[:fileIndex]
    programSearchTerms1.append(te)

print(programSearchTerms1)

['Green+Building', 'Green+Buildings', 'Sustainable+building', 'Sustainable+buildings', 'Environmentally+friendly+design', 'Sustainability', 'Sustainable+development', 'Energy', 'Water', 'Waste', 'Thermal+comfort', 'Electricity', 'Solar', 'Wind+power', 'Green+infrastructure', 'Life+cycle+assessment']
['BIM', 'Building+Information+Modelling', 'Building+information+model', '6D+BIM', 'Digital+construction', 'Smart+construction', 'Digital+engineering']


In [7]:
# https://www.mdpi.com/search?sort=pubdate&page_count=50&advanced=%28%40%28abstract%29a%40%28abstract%29green%29&featured=&subjects=&journals=sustainability&article_types=&countries=
# a%40%28abstract%29green%29&featured=&subjects=&journals=sustainability&article_types=&countries=

In [8]:
baseUrl = 'https://www.mdpi.com/search?sort=pubdate&page_count=10&advanced=%28%40%28abstract%29'
querySet = []

for item1 in programSearchTerms1:
    for item2 in programSearchTerms2:
        row = 0
        query = baseUrl + item1 +"%40%28abstract%29"+ item2+"%29&featured=&subjects=&journals=buildings&article_types=&countries="
        print(query)
        response = requests.get(query)
        soup = BeautifulSoup(response.text, 'html.parser')
        h1 = soup.findAll('div',{'class': ['content__container content__container--overflow-initial'] })

        # Find number of search results
        for h in h1:
            noOfSearches = re.search(r'\((.*?)\)', h.find_next('h1').text).group(1)
            print(noOfSearches)
            if int(noOfSearches) > 10:
                page = int(noOfSearches) // 10
                lastPage = int(noOfSearches) % 10
                if lastPage > 0:
                    page = page + 1

                pageSize = page
            else:
                pageSize = 2
            print("pageSize: ",(pageSize-1))
            
            for currentPage in range(pageSize):
                if currentPage > 0:
                    currQuery = 'https://www.mdpi.com/search?sort=pubdate&page_no='+str(currentPage+1)+'&page_count=10&year_from=1996&year_to=2021&journals=buildings&advanced=%28%40%28abstract%29'+item1+'%40%28abstract%29'+item2+'%29&view=default'
                else:
                    currQuery = query
                print(currQuery)
                
                response = requests.get(currQuery)
                soup = BeautifulSoup(response.text, 'html.parser')
                allLinks = soup.find(['div'],{'class':['jscroll']})
                allDivs = allLinks.findAll(['div'],{'class':['generic-item article-item']})                                                        
                print(len(allDivs))
                
                
                for currLink in allDivs:
                    aa = currLink.find(['div'],{'class':['article-content']})                    
#                     print(aa.find('a',{'class':['title-link']}).get('href'))
                    
                    
#                 for currLink in allLinks:
#                     print(len(allLinks))
                    print("Reading row: ",row,"/",noOfSearches)
                    df.loc[row,'searchTerm1'] = item1
                    df.loc[row,'searchTerm2'] = item2
#                     print(currLink.find('a'))
                    ext = currLink.find('a',{'class':['title-link']}).get('href')  
#                     print("before: ",ext)
                    if not ext is None:
#                         ext = ext[:len(ext)-4]
#                         print("after: ",ext)
                        link = 'https://www.mdpi.com' + ext + '/htm'
                        df.loc[row,'url'] = link
                        print(link)
                        linkResponse = requests.get(link)
                        linkSoup = BeautifulSoup(linkResponse.text, 'html.parser')

                        # Title                         
                        titleSoup = linkSoup.find(['h1'],{'class':['title hypothesis_container']})

                        if titleSoup is not None:
                            title = titleSoup.text
                            print(title)
                            df.loc[row,'Title'] = title
                        else:
                            df.loc[row,'Title'] = 'None'

                        # Abstract Title                         
                        absTitle = linkSoup.findAll(['div'],{'class':['html-abstract-title']})
                        
                        if absTitle is not None:
                            # Finding Abstract title                 
                            for abst in absTitle:
                                df.loc[row,'section0'] = abst.text.strip()                                                        
                        else:
                            df.loc[row,'section0'] = 'None'   

                        # Abstract body
                        
                        # Finding Abstract body                 
                        absData = linkSoup.findAll(['div'],{'class':['art-abstract in-tab hypothesis_container']})
                        
                        if absData is not None:                            
                            # Abstract data                 
                            for a in absData:
                                df.loc[row,'sectionData0'] = a.text
                        else:
                            df.loc[row,'sectionData0'] = "None"

                        # Whole body                             
                        wholeBody = linkSoup.find(['div'],{'class':['html-body']})
                        
                        count = 0
                        
                        if wholeBody is not None:                        
                            for value in (wholeBody):
                                sNo = 'section' + str(count + 1)

                                df.loc[row,sNo] = value.find_next('section').find_next('h2').text
                                sData = 'sectionData'+ str(count + 1)
                                if value.find_next('section').find_next('div',{'class':'html-p'}) is not None:
                                    df.loc[row,sData] = value.find_next('section').find_next('div',{'class':'html-p'}).text
                                else:
                                    df.loc[row,sData] = 'None'
                                count = count + 1
                        else:
                            pass

                    row = row + 1

https://www.mdpi.com/search?sort=pubdate&page_count=10&advanced=%28%40%28abstract%29BIM%40%28abstract%29Green+Building%29&featured=&subjects=&journals=buildings&article_types=&countries=
15
pageSize:  1
https://www.mdpi.com/search?sort=pubdate&page_count=10&advanced=%28%40%28abstract%29BIM%40%28abstract%29Green+Building%29&featured=&subjects=&journals=buildings&article_types=&countries=
10
Reading row:  0 / 15
https://www.mdpi.com/2071-1050/13/7/3675/htm

Optimizing Energy Use, Cost and Carbon Emission through Building Information Modelling and a Sustainability Approach: A Case-Study of a Hospital Building 
Reading row:  1 / 15
https://www.mdpi.com/2076-3417/11/5/2167/htm

Interoperability between Building Information Modelling (BIM) and Building Energy Model (BEM) 
Reading row:  2 / 15
https://www.mdpi.com/2071-1050/12/23/9988/htm

Assessing the Effectiveness of Building Information Modeling in Developing Green Buildings from a Lifecycle Perspective 
Reading row:  3 / 15
https://www.m

4
Reading row:  0 / 4
https://www.mdpi.com/1996-1073/14/11/3127/htm

Use of BIM-FM to Transform Large Conventional Public Buildings into Efficient and Smart Sustainable Buildings 
Reading row:  1 / 4
https://www.mdpi.com/1996-1073/13/3/586/htm

Envelope Thermal Performance Analysis Based on Building Information Model (BIM) Cloud Platform—Proposed Green Mark Collaboration Environment 
Reading row:  2 / 4
https://www.mdpi.com/2079-9292/8/7/724/htm

Building Information Management (BIM) and Blockchain (BC) for Sustainable Building Design Information Management Framework 
Reading row:  3 / 4
https://www.mdpi.com/2071-1050/11/7/1896/htm

Investigating the Constraints to Buidling Information Modeling (BIM) Applications for Sustainable Building Projects: A Case of China 
https://www.mdpi.com/search?sort=pubdate&page_no=2&page_count=10&year_from=1996&year_to=2021&journals=buildings&advanced=%28%40%28abstract%29BIM%40%28abstract%29Sustainable+buildings%29&view=default
0
https://www.mdpi.com/sea


Green BIM Assessment Applying for Energy Consumption and Comfort in the Traditional Public Market: A Case Study 
Reading row:  35 / 46
https://www.mdpi.com/2071-1050/11/16/4398/htm

Bibliometric Maps of BIM and BIM in Universities: A Comparative Analysis 
Reading row:  36 / 46
https://www.mdpi.com/2071-1050/11/14/3973/htm

A Semantic and Social Approach for Real-Time Green Building Rating in BIM-Based Design 
Reading row:  37 / 46
https://www.mdpi.com/2079-9292/8/6/599/htm

A Building Information Modelling (BIM) based Water Efficiency (BWe) Framework for Sustainable Building Design and Construction Management 
Reading row:  38 / 46
https://www.mdpi.com/2071-1050/11/1/286/htm

Multidisciplinary Optimization of Life-Cycle Energy and Cost Using a BIM-Based Master Model 
Reading row:  39 / 46
https://www.mdpi.com/2071-1050/10/11/3831/htm

Regional Comparisons of Contemporary Construction Industry Sustainable Concepts in the Chinese Context 
https://www.mdpi.com/search?sort=pubdate&page_no


Virtual Fence System Based on IoT Paradigm to Prevent Occupational Accidents in the Construction Sector 
https://www.mdpi.com/search?sort=pubdate&page_no=2&page_count=10&year_from=1996&year_to=2021&journals=buildings&advanced=%28%40%28abstract%29BIM%40%28abstract%29Energy%29&view=default
10
Reading row:  10 / 107
https://www.mdpi.com/2071-1050/13/13/7155/htm

Evaluation of Energy Performance and Comfort: Case-Study of University Buildings with Design Adapted to Local Climate 
Reading row:  11 / 107
https://www.mdpi.com/2075-5309/11/6/246/htm

BIM-Based Energy Analysis and Sustainability Assessment—Application to Portuguese Buildings 
Reading row:  12 / 107
https://www.mdpi.com/2071-1050/13/12/6576/htm

Machine Learning for the Improvement of Deep Renovation Building Projects Using As-Built BIM Models 
Reading row:  13 / 107
https://www.mdpi.com/1996-1073/14/10/2956/htm

A Procedure for Automating Energy Analyses in the BIM Context Exploiting Artificial Neural Networks and Transfer Lea


A BIM-Based Framework and Databank for Reusing Load-Bearing Structural Elements 
Reading row:  54 / 107
https://www.mdpi.com/2071-1050/12/7/2633/htm

Uncertainty Analysis of Embedded Energy and Greenhouse Gas Emissions Using BIM in Early Design Stages 
Reading row:  55 / 107
https://www.mdpi.com/1996-1944/13/5/1052/htm

Structural Applications of Thermal Insulation Alkali Activated Materials with Reduced Graphene Oxide 
Reading row:  56 / 107
https://www.mdpi.com/2071-1050/12/5/1704/htm

Reduction Strategies for Greenhouse Gas Emissions from High-Speed Railway Station Buildings in a Cold Climate Zone of China 
Reading row:  57 / 107
https://www.mdpi.com/1424-8220/20/1/139/htm

A Visual and Persuasive Energy Conservation System Based on BIM and IoT Technology 
Reading row:  58 / 107
https://www.mdpi.com/2071-1050/11/22/6473/htm

Energy-Efficient Window Retrofit for High-Rise Residential Buildings in Different Climatic Zones of China 
Reading row:  59 / 107
https://www.mdpi.com/2076-341

...

0
pageSize:  1
https://www.mdpi.com/search?sort=pubdate&page_count=10&advanced=%28%40%28abstract%296D+BIM%40%28abstract%29Environmentally+friendly+design%29&featured=&subjects=&journals=buildings&article_types=&countries=
0
https://www.mdpi.com/search?sort=pubdate&page_no=2&page_count=10&year_from=1996&year_to=2021&journals=buildings&advanced=%28%40%28abstract%296D+BIM%40%28abstract%29Environmentally+friendly+design%29&view=default
0
https://www.mdpi.com/search?sort=pubdate&page_count=10&advanced=%28%40%28abstract%296D+BIM%40%28abstract%29Sustainability%29&featured=&subjects=&journals=buildings&article_types=&countries=
1
pageSize:  1
https://www.mdpi.com/search?sort=pubdate&page_count=10&advanced=%28%40%28abstract%296D+BIM%40%28abstract%29Sustainability%29&featured=&subjects=&journals=buildings&article_types=&countries=
1
Reading row:  0 / 1
https://www.mdpi.com/2071-1050/12/6/2436/htm

Sustainability-Based Lifecycle Management for Bridge Infrastructure Using 6D BIM 
https://www.mdpi.

0
https://www.mdpi.com/search?sort=pubdate&page_count=10&advanced=%28%40%28abstract%29Digital+construction%40%28abstract%29Sustainable+building%29&featured=&subjects=&journals=buildings&article_types=&countries=
0
pageSize:  1
https://www.mdpi.com/search?sort=pubdate&page_count=10&advanced=%28%40%28abstract%29Digital+construction%40%28abstract%29Sustainable+building%29&featured=&subjects=&journals=buildings&article_types=&countries=
0
https://www.mdpi.com/search?sort=pubdate&page_no=2&page_count=10&year_from=1996&year_to=2021&journals=buildings&advanced=%28%40%28abstract%29Digital+construction%40%28abstract%29Sustainable+building%29&view=default
0
https://www.mdpi.com/search?sort=pubdate&page_count=10&advanced=%28%40%28abstract%29Digital+construction%40%28abstract%29Sustainable+buildings%29&featured=&subjects=&journals=buildings&article_types=&countries=
0
pageSize:  1
https://www.mdpi.com/search?sort=pubdate&page_count=10&advanced=%28%40%28abstract%29Digital+construction%40%28abstract

0
pageSize:  1
https://www.mdpi.com/search?sort=pubdate&page_count=10&advanced=%28%40%28abstract%29Digital+construction%40%28abstract%29Green+infrastructure%29&featured=&subjects=&journals=buildings&article_types=&countries=
0
https://www.mdpi.com/search?sort=pubdate&page_no=2&page_count=10&year_from=1996&year_to=2021&journals=buildings&advanced=%28%40%28abstract%29Digital+construction%40%28abstract%29Green+infrastructure%29&view=default
0
https://www.mdpi.com/search?sort=pubdate&page_count=10&advanced=%28%40%28abstract%29Digital+construction%40%28abstract%29Life+cycle+assessment%29&featured=&subjects=&journals=buildings&article_types=&countries=
0
pageSize:  1
https://www.mdpi.com/search?sort=pubdate&page_count=10&advanced=%28%40%28abstract%29Digital+construction%40%28abstract%29Life+cycle+assessment%29&featured=&subjects=&journals=buildings&article_types=&countries=
0
https://www.mdpi.com/search?sort=pubdate&page_no=2&page_count=10&year_from=1996&year_to=2021&journals=buildings&advan

0
pageSize:  1
https://www.mdpi.com/search?sort=pubdate&page_count=10&advanced=%28%40%28abstract%29Smart+construction%40%28abstract%29Waste%29&featured=&subjects=&journals=buildings&article_types=&countries=
0
https://www.mdpi.com/search?sort=pubdate&page_no=2&page_count=10&year_from=1996&year_to=2021&journals=buildings&advanced=%28%40%28abstract%29Smart+construction%40%28abstract%29Waste%29&view=default
0
https://www.mdpi.com/search?sort=pubdate&page_count=10&advanced=%28%40%28abstract%29Smart+construction%40%28abstract%29Thermal+comfort%29&featured=&subjects=&journals=buildings&article_types=&countries=
0
pageSize:  1
https://www.mdpi.com/search?sort=pubdate&page_count=10&advanced=%28%40%28abstract%29Smart+construction%40%28abstract%29Thermal+comfort%29&featured=&subjects=&journals=buildings&article_types=&countries=
0
https://www.mdpi.com/search?sort=pubdate&page_no=2&page_count=10&year_from=1996&year_to=2021&journals=buildings&advanced=%28%40%28abstract%29Smart+construction%40%28ab

0
https://www.mdpi.com/search?sort=pubdate&page_count=10&advanced=%28%40%28abstract%29Digital+engineering%40%28abstract%29Sustainable+development%29&featured=&subjects=&journals=buildings&article_types=&countries=
0
pageSize:  1
https://www.mdpi.com/search?sort=pubdate&page_count=10&advanced=%28%40%28abstract%29Digital+engineering%40%28abstract%29Sustainable+development%29&featured=&subjects=&journals=buildings&article_types=&countries=
0
https://www.mdpi.com/search?sort=pubdate&page_no=2&page_count=10&year_from=1996&year_to=2021&journals=buildings&advanced=%28%40%28abstract%29Digital+engineering%40%28abstract%29Sustainable+development%29&view=default
0
https://www.mdpi.com/search?sort=pubdate&page_count=10&advanced=%28%40%28abstract%29Digital+engineering%40%28abstract%29Energy%29&featured=&subjects=&journals=buildings&article_types=&countries=
1
pageSize:  1
https://www.mdpi.com/search?sort=pubdate&page_count=10&advanced=%28%40%28abstract%29Digital+engineering%40%28abstract%29Energy%2

In [9]:
df['journalName'] = 'buildings'

In [10]:
df.to_csv('kgRawData/kg-mdpi-buildings.csv')

In [11]:
import pandas as pd

pd.read_csv('kgRawData/kg-mdpi-buildings.csv')

,Unnamed: 0,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,...,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19,journalName
0,0,Digital+engineering,Energy,https://www.mdpi.com/1996-1073/13/8/2087/htm,\nApproaches for Modelling the Physical Behavi...,Abstract,\nModels of technical systems are an essential...,1. Introduction,"Today, the general consensus is that the early...",2. Abstract Physics—Background,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,buildings
1,1,Smart+construction,Energy,https://www.mdpi.com/1996-1073/13/9/2190/htm,\nSmart Building Integration into a Smart City...,Abstract,\nThe aim of this study is to define the featu...,1. Introduction,According to the 2018 Revision of World Urbani...,2. Smart City and Smart Building: Concept Evo...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,buildings
2,2,Smart+construction,Energy,https://www.mdpi.com/2077-1312/7/4/83/htm,\nConstructing Governance Framework of a Green...,Abstract,\nDeveloping a green and smart port is a signi...,1. Introduction,A port is not only an important platform and n...,2. Literature Review,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,buildings
3,3,Smart+construction,Sustainable+development,https://www.mdpi.com/2071-1050/10/9/3348/htm,\nEvaluation on Construction Level of Smart Ci...,Abstract,"\nCurrently, the construction of smart cities ...",1. Introduction,The world is at an unprecedented level of urba...,2. Establishment of Indicator System,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,buildings
4,4,Building+information+model,Energy,https://www.mdpi.com/1996-1073/13/11/2967/htm,\nDistrict Energy Systems: Challenges and New ...,Abstract,\nThe change from a centralized to a decentral...,1. Introduction & Motivation,In order to cope with the challenges posed by ...,2. The Energetic Neighborhood District ENaQ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,buildings
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,102,BIM,Energy,https://www.mdpi.com/1996-1073/9/1/50/htm,\nAn Algorithm to Translate Building Topology ...,Abstract,\nThis paper presents an algorithm to translat...,1. Introduction,The complex data exchange between architectura...,2. Background,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,buildings
103,103,BIM,Energy,https://www.mdpi.com/2075-5309/5/4/1361/htm,\nGuidelines for Using Building Information Mo...,Abstract,"\nBuilding energy modeling (BEM), a subset of ...",1. Introduction,As sustainability increasingly becomes a stand...,2. Literature Review,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,buildings
104,104,BIM,Energy,https://www.mdpi.com/2075-5309/5/4/1156/htm,\nAssessing Embodied Energy and Greenhouse Gas...,Abstract,\nGreenhouse gas (GHG) emissions from construc...,1. Introduction,Construction-related energy use and associated...,2. Previous Research,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,buildings
105,105,BIM,Energy,https://www.mdpi.com/2075-5309/5/1/100/htm,\nBuilding Information Modelling for Smart Bui...,Abstract,\nBuilding information modelling (BIM) provide...,1. Introduction,Computer-aided design (CAD) techniques have be...,2. Literature Review,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,buildings
